In [11]:
import json
import sys
sys.path.append('../src')
from pydantic import BaseModel

# Define the Pydantic model
class ExperimentConfig(BaseModel):
    epochs: int
    learning_rate: float
    batch_size: int

# Path to your JSON file
json_file_path = '/home/xmootoo/Documents/VSCode/ssl-seizure-detection/exp/1.0/test.json'

# Load and parse the JSON file
with open(json_file_path, 'r') as file:
    data = json.load(file)

# Assuming you want to load the configuration for 'experiment_1'
experiment_data = data['experiment_1']

# Create an instance of the ExperimentConfig with the loaded data
config = ExperimentConfig(**experiment_data)

# config now contains the configuration from the JSON file
print(config)

epochs=10 learning_rate=0.001 batch_size=32


In [2]:
import submitit
import os

def run_training(x):

    # Environment setup
    wandb_api_key = os.environ.get("WANDB_API_KEY")
    if not wandb_api_key:
        raise ValueError("WANDB_API_KEY not set in environment variables")
    
    module_load = "module load cuda/11.7 cudnn/8.9.5.29 python/3.10"
    activate_env = "source ~/torch2_cuda11.7/bin/activate"

    # Define the command to run, including module load and environment activation
    command = f"{module_load} && {activate_env} && python test.py {x} " 

    os.system(command)

def submit_job():
    # Create an executor for SLURM
    logdir="/home/xmootoo/projects/def-milad777/xmootoo/ssl_epilepsy/jobs"
    executor = submitit.AutoExecutor(folder=logdir)

    # Set SLURM job parameters
    executor.update_parameters(
        gpus_per_node=1,
        tasks_per_node=1,
        cpus_per_task=4,
        mem_gb=12 * 4,  # Adjust memory as needed
        time="00:25:00",
        slurm_partition="def-milad777",  # Replace with your SLURM partition
        slurm_array_parallelism=1
    )

    # Submit the job
    job = executor.submit(run_training, 4)
    print(f"Job submitted: {job.job_id}")

if __name__ == "__main__":
    submit_job()